In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# --- Setup ---
DATA_PATH = "/mnt/big_gulp/nc4_rat_data/Maze_Rats"
rat_id = 19
rat_dir = f"NC400{rat_id}"
session_dates = sorted(os.listdir(os.path.join(DATA_PATH, rat_dir)))  # e.g., ['250428', '250429']

# Where the cumulative analysis file is stored
summary_csv_path = os.path.join(DATA_PATH, rat_dir, "Processed", "cumulative_summary.csv")

# Load existing summary if it exists
if os.path.exists(summary_csv_path):
    df_summary = pd.read_csv(summary_csv_path)
    analyzed_dates = set(df_summary["session_date"].astype(str))
    print(f"Already analyzed: {sorted(analyzed_dates)}")
else:
    df_summary = pd.DataFrame()
    analyzed_dates = set()

In [ ]:
# Collect new sessions
new_summaries = []

for date in session_dates:
    if date in analyzed_dates:
        continue

    session_csv_path = os.path.join(DATA_PATH, rat_dir, date, "Processed", "session_data", "trials_df.csv")
    if not os.path.exists(session_csv_path):
        continue

    df = pd.read_csv(session_csv_path)
    percent_correct = df["correct"].mean() * 100
    avg_response_time = df["response_time"].mean()
    summary = {
        "rat_id": rat_id,
        "session_date": date,
        "n_trials": len(df),
        "percent_correct": percent_correct,
        "avg_response_time": avg_response_time,
    }
    new_summaries.append(summary)
    print(f"Added {date}: {percent_correct:.1f}% correct, {avg_response_time:.2f}s RT")

# Update summary dataframe
if new_summaries:
    df_summary = pd.concat([df_summary, pd.DataFrame(new_summaries)], ignore_index=True)
    df_summary = df_summary.sort_values(by="session_date")
    df_summary.to_csv(summary_csv_path, index=False)
    print(f"Updated summary saved to: {summary_csv_path}")
else:
    print("No new sessions to add.")

In [ ]:
# --- Plotting ---
if not df_summary.empty:
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=df_summary, x="session_date", y="percent_correct", marker="o")
    plt.title(f"Percent Correct Over Time (Rat {rat_id})")
    plt.xticks(rotation=45)
    plt.ylim(0, 100)
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(10, 5))
    sns.lineplot(data=df_summary, x="session_date", y="avg_response_time", marker="o")
    plt.title(f"Average Response Time Over Time (Rat {rat_id})")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("No data available to plot.")